# **Modélisation statistique**
Nous allons mener des régressions linéaires de la production électrique éolienne (données API RTE) sur les données météorologiques de vents (données API ...)

Dans un premier temps, nous allons créer un data frame qui contiendra la moyenne nationale de la production électrique éolienne ainsi que la moyenne des vents nationaux, heure par heure pour l'année 2021.
Ensuite, nous affinerons l'analyse en faisant des régressions par région/année.

### **1) Récupération des données**
#### **a) Données météorologiques**

In [170]:
import pandas as pd

donnees_vent = pd.read_csv(r'C:\Users\peign\Desktop\ProjetPython\donnees_vent.csv', sep=",", names=['Index', 'Time', 'Windspeed'])

In [173]:
donnees_vent.head()

,Index,Time,Windspeed
0,0,"[2011-04-2300:00:00,2011-04-2301:00:00,2011-04...","[2.34, 2.33, 1.8, 1.84, 1.7, 2.01, 1.35, 1.08,..."
1,1,"[2014-04-0100:00:00,2014-04-0101:00:00,2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
2,2,"[2014-04-0100:00:00,2014-04-0101:00:00,2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
3,3,"[2014-04-0100:00:00,2014-04-0101:00:00,2014-04...","[0.86, 1.2, 1.66, 1.21, 1.51, 2.02, 2.51, 2.81..."
4,4,"[2021-11-0400:00:00,2021-11-0401:00:00,2021-11...","[1.61, 1.71, 1.53, 1.86, 1.82, 1.57, 1.61, 1.7..."


In [176]:
donnees_vent["Time"][120]

'2018-12-2000:00:00,2018-12-2001:00:00,2018-12-2002:00:00,2018-12-2003:00:00,2018-12-2004:00:00,2018-12-2005:00:00,2018-12-2006:00:00,2018-12-2007:00:00,2018-12-2008:00:00,2018-12-2009:00:00,2022-12-0414:00:00,2022-12-0415:00:00,2022-12-0416:00:00,2022-12-0417:00:00,2022-12-0418:00:00,2022-12-0419:00:00,2022-12-0420:00:00,2022-12-0421:00:00,2022-12-0422:00:00,2022-12-0423:00:00'

In [175]:
caracteres = ["\n","DatetimeIndex(", "               "," ","...","'","[ns]"]
    
for string in caracteres:
    donnees_vent["Time"] = donnees_vent["Time"].apply(lambda x: x.replace(string, ''))
    
donnees_vent["Time"] = donnees_vent["Time"].str.extract(r"\[(.+?)\]")

In [165]:
donnees_vent["Time"] = donnees_vent["Time"].tolist()

#### **b) Données RTE**

### **2) Régression au niveau national**
#### **Calcul des moyennes**

In [ ]:
hours = df.groupby(df.index.hour)

mean_nat = hours.mean()

#### **Régression linéaire**

In [ ]:
import statsmodels.formula.api as smf

def regression_results(data):
    model = smf.ols("electricity ~ wind", data=data).fit()
    predictions = model.predict(data)
    print(model.summary())
    
regression_results(mean_nat)  

#### **Graphiques**

In [ ]:
def plot_linear_regression(data):
    plot = sns.regplot(x="wind", y="electricity", data=data)
    sns.lmplot(x="wind", y="electricity", data=data)
    return plot

plot = plot_linear_regression(mean_nat)
plt.show()